# Demonstration of awsgnssroutils.database

Authors: Amy McVey (amcvey@aer.com) and Stephen Leroy (sleroy@aer.com) \
Version: 1.2.1 \
Date: 26 September 2023

This notebook demonstrates the functionality of the awsgnssroutils.database 
module, which contains utilities to query the AWS Registry of Open Data repository
of GNSS radio occultation data. It does so using database files posted in the
AWS repository. Specifically, the module defines two classes: *RODatabaseClient* and 
*OccList*. The former creates a gateway to the metadata on GNSS radio occultation 
(RO) soundings hosted in the AWS Registry of Open Data and enables a local mirror 
repository of that metadata if one is desired. The latter creates instances of 
lists of RO soundings and their associated metadata and offers several methods to 
manipulate those metadata and even download RO data files. 


## Functionality
This module defines two classes: RODatabaseClient and OccList. The first 
creates a portal to a database of RO metadata, and the second is an instance 
of a list of radio occultations (ROs). Each are described below. It is mandatory 
that a new user initialize a repository that tracks the history of prior searches. 
Doing so is necessary to accelerate the querying of RO metadata. **The user will 
encounter long search times the first time a segment of the database is queries. 
With the local repository of previous searches, subsequent queries along the same 
segment will be much shorter.** 


### Initialization

To simplify the user's life, define your defaults using the setdefaults function. You will be able to define default paths for storing occultation metadata, RO data files, and the AWS RO processing version. The local storage of metadata will enable greatly accelerated queries. 

In [ ]:
from awsgnssroutils.database import setdefaults
import os

HOME = os.path.expanduser("~")
repository_directory = os.path.join( HOME, "local/rodatabase" )
rodata_root_directory = os.path.join( HOME, "Data/rodata" )

setdefaults( repository=repository_directory, rodata=rodata_root_directory, version="v1.1" )

The *repository_directory* is a path to the directory where a collection of json 
files will be downloaded and stored. It must be an absolute path. The 
*rodata_root_directory* is the default root directory for RO data downloads. It 
is optional, but it must also be an absolute path if it is specified. 

*Be certain you have enough storage available in your ~/local/rodatabase 
file space. RO metadata requires ~50 GB of storage. If you don't 
have enough space allocated, edit the above command to point toward a 
repository directory on a scratch volume, which typically has plenty of 
storage available for all RO metadata.*

You can prepopulate all RO metadata using the awsgnssroutils.databaes.populate 
function. Pre-populating is unnecessary, but it will make all queries run very 
efficiently regardless of search history. Pre-populating should only be done 
intermittently, at a cadence no more frequent than monthly. If building a metadata 
database from scratch, pre-populating can take ~10 minutes or more. 

In [ ]:
from awsgnssroutils.database import populate
populate()

### RODatabaseClient: 
Create an instance of a portal to the metadata database of all RO data in the AWS 
Registry of Open Data. For example,

In [ ]:
from awsgnssroutils.database import RODatabaseClient

rodb = RODatabaseClient( update=False )
print( rodb )

creates a database interface with a local repository of the 
metadata in the directory defined at initialization. The local repository 
will be used for querying RO metadata: future queries first consult the 
local repository, and if the metadata does not already exist in the local 
repository, the metadata is retrieved and copied into the local repository. 
Creating and accessing a local repository is far more efficient than 
not creating a local repository. First queries of metadata will still be 
slow, but subsequent queries of the same metadata are made over an order of 
magnitude faster by reading from the local file system&mdash;the local 
repository&mdash;rather than by reading directly from the AWS S3 bucket. 

By specifying **update** as True, the local repository is updated at the 
instantiation of rodb. The update compares metadata in the repository 
of metadata on the local file system to the same metadata files in the
AWS Registry of Open Data and updates the local metadata as needed. 
The update does not add any "new" metadata files to the local repository. 
(The default is False.) An update can be computationally expensive, however. 
We recommend executing an update=True instantiations of RODatabaseClient 
only periodically. The contents of the metadatabase files in the AWS 
Registry of Open Data rarely change, and so it should not be necessary to 
update the local mirror at every instantiation of RODatabaseClient. 

There are two methods to create a list of occultations through the 
database client. One is to perform an inquiry in which missions and/or 
a date-time range is specified, and a second is to restore a previously 
saved list of RO data. 

In [ ]:
occlist = rodb.query( missions="champ" )
print( occlist )

generates an OccList containing metadata on all CHAMP RO data. The inquiry 
can be performed instead over a range in time. The date-time fields are 
always ISO format times...

In [ ]:
occlist = rodb.query( datetimerange=("2019-06-01","2019-07-01") )
print( occlist )

creates an OccList of metadata for all RO soundings in the month of June, 
2019, regardless of mission. 

The other option to create an OccList is by restoring a previously 
saved OccList: 

In [ ]:
savefile = "old_occlist.json"
occlist.save( savefile )
occlist1 = rodb.restore( savefile )
print( occlist1 )

in which the old OccList was saved in a JSON format file. 

### OccList:

An instance of the class OccList is contains the metadata on a list of RO 
soundings along with pointers to the RO data files in the AWS Registry of 
Open Data S3 bucket. AWS functionality is completely embedded in the 
methods of the OccList class. Those methods include the ability to 
subset/filter the list according to geolocation and time, 
GNSS transmitter/constellation, GNSS receiver, whether it is a rising or a 
setting occultation, etc. It also includes the ability to combine 
instances of OccList, save the OccList to a JSON format file for future 
restoration by RODatabaseClient.restore, and even download RO data files. 

In order to filter an OccList previously generated by 
RODatabaseClient.query or RODatabaseClient.restore, use the OccList.filter 
method: 

In [ ]:
champoccs = rodb.query( missions="champ" )
print( champoccs )
champoccs2003 = champoccs.filter( datetimerange=("2003-01-01","2004-01-01") )
print( champoccs2003 )

illustrates how to apply a filter in date-time, retaining all CHAMP RO 
metadata for the year 2003. Filtering can be done in longitude and latitude 
as well: 

In [ ]:
champoccs_US = champoccs.filter( longituderange=(-110,-70), latituderange=(25,55) )
print( champoccs_US )

and even those can be subset by local time (a.k.a. solar time): 

In [ ]:
champoccs_US_midnight = champoccs_US.filter( localtimerange=(22,2) )
print( champoccs_US_midnight )

in which the local time range is given in hours and can wrap around 
midnight. Other filter options are for the GNSS constellation used as 
transmitters ("G" for GPS, "R" for GLONASS, "E" for Galileo, "C" for 
BeiDou), for individual transmitters ("G01", etc.), for individual 
receivers ("cosmic1c1", "metopb", etc.), and for occultation 'geometry' 
("rising" vs. "setting"). 

One can get information on the metadata in an OccList using the 
OccList.info method. For instance, if you want to get a listing of all of 
the Spire receiver satellites, do 

In [ ]:
spire = rodb.query( missions="spire" )
print( f"Number of Spire occultation found: {spire.size}" )
spire_receivers = spire.info( "receiver" )
print( spire_receivers )

Notice that OccList.size contains a count of the number of RO entries in this 
instance of OccList. The first step in this process could be time consuming if the Spire 
metadata do not already reside on the local file system and the rodb object 
does not interface with a local repository. One can also get a list of the 
GNSS transmitters tracked by Spire on a particular day by 

In [ ]:
spire_day = spire.filter( datetimerange=("2021-12-01","2021-12-02") )
spire_day_transmitters = spire_day.info("transmitter")
print( spire_day_transmitters )

which will give a list of all GNSS transmitters tracked by all Spire 
satellites on December 1, 2021. The spire_day list can be split up between 
rising and setting RO soundings as well: 

In [ ]:
spire_day_rising = spire_day.filter( geometry="rising" )
print( "spire_day_rising = " + str(spire_day_rising) )
spire_day_setting = spire_day.filter( geometry="setting" )
print( "spire_day_setting = " + str(spire_day_setting) )

Another filter option is by the availability of various types of AWS RO data file types. 
There are three AWS-native RO data file types (AWSfiletypes): *calibratedPhase*, which contains precise 
orbits, calibrated phase, and amplitude measurements; *refractivityRetrieval*, which contains 
retrieved bending angles, impact parameters, and refractivity profiles; and *atmosphericRetrieval*, 
which contains retrievals of atmospheric quantities such as temperature, pressure, and water 
vapor partial pressure. The filetype has format "{center}\_{AWSfiletype}", where the "center" is 
one of the contributing RO processing centers ("ucar", "jpl", "romsaf"). 

In [ ]:
spire_day_with_bending_data = spire_day.filter( availablefiletypes="ucar_refractivityRetrieval" )
print( "spire_day_with_bending_data = " + str(spire_day_with_bending_data) )

Then it is possible to save the spire metadata OccList to a JSON file 
for future restoration by 

In [ ]:
spire.save( "spire_metadata.json" )

The metadata contain pointers to the RO sounding data files in the 
AWS Open Data bucket. To get information on the data files available, 
use the `OccList.info( "filetype" )` method. For example, to find out the 
types of RO data files avialable for the month of June, 2009: 

In [ ]:
from pprint import pprint

June2009 = rodb.query( datetimerange=("2009-06-01","2009-06-30") )
filetype_dict = June2009.info( "filetype" )
pprint( filetype_dict, indent=2 )

which will return a dictionary with the AWS-native RO file types as keys 
with corresponding values being the counts of each. The file types have the 
format "{processing_center}_{file_type}" in which "processing_center" is an 
RO processing center that contributed to the AWS repository ("ucar", 
"romsaf", "jpl") and the "file_type" is one of "calibratedPhase", 
"refractivityRetrieval", or "atmosphericRetrieval". 

The values of the longitude, latitude, datetime, and localtimes of the RO 
soundings in an OccList can be obtained using the OccList.values() method: 

In [ ]:
longitudes = June2009.values( "longitude" )
latitudes = June2009.values( "latitude" )
localtimes = June2009.values( "localtime" )

where each of these variables being a masked numpy ndarray. 

Finally, RO data files themselves can be downloaded for subsequent 
scientific analysis using the OccList.download() method. If one wishes to 
download the all RO bending angle data contributed by the ROMSAF to the archive 
for the day of June 5, 2012, one only need execute the commands  

In [ ]:
day_list = rodb.query( datetimerange=("2012-06-05T00:00:00","2012-06-05T23:59:59") )
files = day_list.download( "romsaf_refractivityRetrieval" )
pprint( files )

which will download all file type "refractivityRetrieval" contributed by 
the ROM SAF into the rodata_directory specified at initialization. The user 
can override the download directory using the **rodata** keyword, in which 
case the user can also use the **keep_aws_structure=False** declaration to 
glob all files into that directory without the directory hierarchy of the 
AWS RO repository. The download method does require the download directory 
be specified either in the resources file defined at initialization or as 
specified by keyword in calling the download method. 

## Exercise
Plot the geolocation of all GNSS RO soundings on June 1, 2012. 

In [ ]:
import matplotlib.pyplot as plt
import cartopy.crs as ccrs

plt.clf()
cmap = plt.get_cmap( "nipy_spectral" )
ax = plt.axes( projection=ccrs.PlateCarree() )
ax.coastlines( color="black" )

occlist = rodb.query( datetimerange=("2012-06-01","2012-06-02") )
print( f"Found {occlist.size} soundings." )
missions = occlist.info( "mission" )

for imission, mission in enumerate(missions): 
    occlist_by_mission = occlist.filter( missions=mission )
    lons = occlist_by_mission.values( "longitude" )
    lats = occlist_by_mission.values( "latitude" )
    color = cmap( (imission+0.5)/len(missions) )
    ax.scatter( lons, lats, color=color, marker="o", s=0.2 , label=mission )
    
ax.legend(loc="lower left", fontsize="x-small")
plt.show()